# Cube analysis with Gammapy (part 2)

## Introduction 

In this tutorial we will learn how to compute a morphological and spectral fit simultanously.

This is part 2 of 2 for IACT cube analysis. If you haven't prepared your cubes, PSF and EDISP yet, do part 1 first.

The fitting is done using [Sherpa](http://cxc.harvard.edu/contrib/sherpa47/).

We will use the following classes:

- [gammapy.cube](http://docs.gammapy.org/en/latest/cube/index.html) where are strored the counts, the background, the exposure and the mean psf of this Crab dataset.
- [gammapy.irf.EnergyDispersion](http://docs.gammapy.org/en/latest/api/gammapy.irf.EnergyDispersion.html) where is stored the mean rmf of this Crab dataset.
-  the method [gammapy.cube.SkyCube.to_sherpa_data3d](http://docs.gammapy.org/en/latest/api/gammapy.cube.SkyCube.html#gammapy.cube.SkyCube.to_sherpa_data3d) to transform the counts cube in Sherpa object.
- [gammapy.cube.CombinedModel3DInt](http://docs.gammapy.org/en/latest/api/gammapy.cube.CombinedModel3DInt.html) to combine the spectral and spatial model for the fit if you consider a perfect energy resolution
- [gammapy.cube.CombinedModel3DIntConvolveEdisp](http://docs.gammapy.org/en/latest/api/gammapy.cube.CombinedModel3DIntConvolveEdisp.html) to combine the spectral and spatial model for the fit taking into account that the true energy is different than the reconstructed one.

We will use the Cubes build on the 4 Crab observations of gammapy-extra. You could use the Cubes we just created with the notebook cube_analysis.ipynb

## Import

In [25]:
import numpy as np
from astropy.io import fits
from astropy.coordinates import SkyCoord, Angle
from regions import CircleSkyRegion
from gammapy.datasets import gammapy_extra
from gammapy.irf import EnergyDispersion
from gammapy.utils.testing import requires_dependency, requires_data
from gammapy.cube import SkyCube, CombinedModel3DInt, CombinedModel3DIntConvolveEdisp, NormGauss2DInt

from sherpa.models import PowLaw1D, TableModel
from sherpa.estmethods import Covariance
from sherpa.optmethods import NelderMead
from sherpa.stats import Cash
from sherpa.fit import Fit

## 3D analysis assuming that there is no energy dispersion (perfect energy resolution)

### Load the different cubes needed for the analysis

We will use the Cubes build on the 4 Crab observations of gammapy-extra. You could use the Cubes we just created with the notebook cube_analysis.ipynb by changing the cube_directory by your local path.

- Counts cube

In [26]:
cube_directory="$GAMMAPY_EXTRA/test_datasets/cube"
filename = cube_directory+"/counts_cube.fits"
counts_3d = SkyCube.read(filename)
#Transformation to a sherpa object
cube = counts_3d.to_sherpa_data3d(dstype='Data3DInt')


- Bakcground Cube

In [27]:
filename = cube_directory+"/bkg_cube.fits"
bkg_3d = SkyCube.read(filename)
bkg = TableModel('bkg')
bkg.load(None, bkg_3d.data.value.ravel())
bkg.ampl = 1
bkg.ampl.freeze()

- Exposure Cube

In [28]:
filename = cube_directory+"/exposure_cube.fits"
exposure_3d = SkyCube.read(filename)
i_nan = np.where(np.isnan(exposure_3d.data))
exposure_3d.data[i_nan] = 0
# In order to have the exposure in cm2 s
exposure_3d.data = exposure_3d.data * 1e4

- PSF Cube

In [29]:
filename = cube_directory+"/psf_cube.fits"
psf_3d = SkyCube.read(filename)

### Setup Combined spatial and spectral model

In [30]:
#Define a 2D gaussian for the spatial model
spatial_model = NormGauss2DInt('spatial-model')
#Define a power law for the spectral model
spectral_model = PowLaw1D('spectral-model')

#Combine spectral and spatial model
coord = counts_3d.sky_image_ref.coordinates(mode="edges")
energies = counts_3d.energies(mode='edges').to("TeV")
#Here the source model will be convolve by the psf: PSF*(source_model*exposure)
source_model = CombinedModel3DInt(coord=coord, energies=energies, use_psf=True, exposure=exposure_3d, psf=psf_3d,
                                      spatial_model=spatial_model, spectral_model=spectral_model)

### Set starting value

In [31]:
center = SkyCoord(83.633083, 22.0145, unit="deg").galactic
source_model.gamma = 2.2
source_model.xpos = center.l.value
source_model.ypos = center.b.value
source_model.fwhm = 0.12
source_model.ampl = 1.0

### Fit

In [32]:
#Define the model
model = bkg + 1E-11 * (source_model)
#Fit to the counts Cube sherpa object
fit = Fit(data=cube, model=model, stat=Cash(), method=NelderMead(), estmethod=Covariance())
result = fit.fit()
err=fit.est_errors()

In [33]:
print(err)

datasets    = None
methodname  = covariance
iterfitname = none
fitname     = neldermead
statname    = cash
sigma       = 1
percent     = 68.2689492137
parnames    = (u'spatial-model.xpos', u'spatial-model.ypos', u'spatial-model.ampl', u'spatial-model.fwhm', 'spectral-model.gamma')
parvals     = (184.19524957441664, -6.1693008203971562, 6.1666646581766011, 0.076340497278248376, 2.3059120375493234)
parmins     = (-0.012831726563911265, -0.012463553183229555, -0.30893010240483937, -0.018826651718713557, -0.059934915469270683)
parmaxes    = (0.012831726563911265, 0.012463553183229555, 0.30893010240483937, 0.018826651718713557, 0.059934915469270683)
nfits       = 0


## Add an exlusion mask for the Fit
For example if you want to exclude a region in the FoV of your cube, you just have to provide a SkyCube with the same dimension than the Counts cube and with 0 in the region you want to exlude and 1 outside. With this SkyCube mask you can select only some energy band for the fit or just some spatial region whatever the energy band or both.

### Define the mask
Here this is a test case, there is no source to exlude in our FOV but we will create a mask that remove some events from the source. You just see at the end that the amplitude fitted in the 3D analysis is lower than the one when you use all the events from the Crab. The principle works even if this is not usefull here, just a testcase.

In [34]:
exclusion_region = CircleSkyRegion(SkyCoord(83.60, 21.88, unit='deg'), Angle(0.1, "deg"))
sky_mask_cube=SkyCube.empty_like(counts_3d)
energies = sky_mask_cube.energies(mode='edges').to("TeV")
coord_center_pix = sky_mask_cube.sky_image_ref.coordinates(mode="center").icrs
lon = np.tile(coord_center_pix.data.lon.degree, (len(energies) - 1, 1, 1))
lat = np.tile(coord_center_pix.data.lat.degree, (len(energies) - 1, 1, 1))
coord_3d_center_pix = SkyCoord(lon, lat, unit="deg")
index_excluded_region = np.where(
        (exclusion_region.center).separation(coord_3d_center_pix) < exclusion_region.radius)
#0 in the region you want to exclude and 1 outside
sky_mask_cube.data.value[:] = 1
sky_mask_cube.data.value[index_excluded_region] = 0
index_region_selected_3d = np.where(sky_mask_cube.data.value == 1)

Add the mask to the data

In [35]:
# Set the counts
cube = counts_3d.to_sherpa_data3d(dstype='Data3DInt')
cube.mask = sky_mask_cube.data.value.ravel()

Select only the background pixels of the Cube of the selected region to create the background model

In [36]:
# Set the bkg and select only the data points of the selected region
bkg = TableModel('bkg')
bkg.load(None, bkg_3d.data.value[index_region_selected_3d].ravel())
bkg.ampl = 1
bkg.ampl.freeze()


Add the indices of the selected region of the Cube to combine the model

In [37]:
# The model is evaluated on all the points then it is compared with the data only on the selected_region
source_model = CombinedModel3DInt(coord=coord, energies=energies, use_psf=True, exposure=exposure_3d, psf=psf_3d,
                                      spatial_model=spatial_model, spectral_model=spectral_model, select_region=True,
                                      index_selected_region=index_region_selected_3d)
# Set starting values
source_model.gamma = 2.2
source_model.xpos = center.l.value
source_model.ypos = center.b.value
source_model.fwhm = 0.12
source_model.ampl = 1.0

# Fit
model = bkg + 1E-11 * (source_model)
fit = Fit(data=cube, model=model, stat=Cash(), method=NelderMead(), estmethod=Covariance())
result = fit.fit()
err=fit.est_errors()

The fitted flux is less than in the previous example since here the mask remove some events from the Crab

In [38]:
print(err)

datasets    = None
methodname  = covariance
iterfitname = none
fitname     = neldermead
statname    = cash
sigma       = 1
percent     = 68.2689492137
parnames    = (u'spatial-model.xpos', u'spatial-model.ypos', u'spatial-model.ampl', u'spatial-model.fwhm', 'spectral-model.gamma')
parvals     = (184.20146538191321, -6.1600047997645975, 5.4193056837212374, 0.08635929788659219, 2.2979723660330258)
parmins     = (-0.018445964718895265, -0.012571297632706941, -0.29632427116507004, -0.021736889307133158, -0.065042149296143173)
parmaxes    = (0.018445964718895265, 0.012571297632706941, 0.29632427116507004, 0.021736889307133158, 0.065042149296143173)
nfits       = 0


## 3D analysis taking into account the energy dispersion

In [39]:
# Set the counts
filename = cube_directory+"/counts_cube.fits"
counts_3d = SkyCube.read(filename)
cube = counts_3d.to_sherpa_data3d(dstype='Data3DInt')

# Set the bkg
filename = cube_directory+"/bkg_cube.fits"
bkg_3d = SkyCube.read(filename)
bkg = TableModel('bkg')
bkg.load(None, bkg_3d.data.value.ravel())
bkg.ampl = 1
bkg.ampl.freeze()

# Set the exposure
filename = cube_directory+"/exposure_cube_etrue.fits"
exposure_3d = SkyCube.read(filename)
i_nan = np.where(np.isnan(exposure_3d.data))
exposure_3d.data[i_nan] = 0
exposure_3d.data = exposure_3d.data * 1e4

# Set the mean psf model
filename = cube_directory+"/psf_cube_etrue.fits"
psf_3d = SkyCube.read(filename)

### Load the mean rmf calculated for the 4 Crab runs

In [40]:
filename = cube_directory+"/rmf.fits"
rmf = EnergyDispersion.read(filename)

In [43]:
# Setup combined spatial and spectral model
spatial_model = NormGauss2DInt('spatial-model')
spectral_model = PowLaw1D('spectral-model')

### Add the mean RMF to the Combine3DInt object 

The model is evaluated on the true energy bin then it is convolved by the energy dispersion to compare to the counts data in reconstructed energy

In [44]:
coord = counts_3d.sky_image_ref.coordinates(mode="edges")
energies = counts_3d.energies(mode='edges').to("TeV")
source_model = CombinedModel3DIntConvolveEdisp(coord=coord, energies=energies, use_psf=True, exposure=exposure_3d,
                                                   psf=psf_3d,
                                                   spatial_model=spatial_model, spectral_model=spectral_model,
                                                   edisp=rmf.data.data)

# Set starting values
center = SkyCoord(83.633083, 22.0145, unit="deg").galactic
source_model.gamma = 2.2
source_model.xpos = center.l.value
source_model.ypos = center.b.value
source_model.fwhm = 0.12
source_model.ampl = 1.0

# Fit
model = bkg + 1E-11 * (source_model)
fit = Fit(data=cube, model=model, stat=Cash(), method=NelderMead(), estmethod=Covariance())
result = fit.fit()
result = fit.fit()
err=fit.est_errors()
print(err)

datasets    = None
methodname  = covariance
iterfitname = none
fitname     = neldermead
statname    = cash
sigma       = 1
percent     = 68.2689492137
parnames    = (u'spatial-model.xpos', u'spatial-model.ypos', u'spatial-model.ampl', u'spatial-model.fwhm', 'spectral-model.gamma')
parvals     = (184.19189525423425, -6.1758238877562386, 6.2283155506945755, 0.071013932890499717, 2.2685809241308674)
parmins     = (-0.011736215022357116, -0.013746467389135797, -0.31228634396780269, -0.020653652043090193, -0.051506896577267751)
parmaxes    = (0.011736215022357116, 0.013746467389135797, 0.31228634396780269, 0.020653652043090193, 0.051506896577267751)
nfits       = 0


Here the Cubes are constructed from dummy data. We don't expect to find the good spectral shape or amplitude since there is a lot of problem with the PFS, rmf etc.. in these dummy data.
On real data, we find the good Crab position and spectral shape for a power law or a power law with an exponential cutoff....